In [1]:
#Import libraries:
import tweepy
import json
import pandas as pd
from sys import argv

In [1]:
import codecs
import re
import string


from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

### Initial nltk installation:
### from ipython: import nltk --> nltk.download() --> select Corpora - wordnet
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import TweetTokenizer

### Initial gensim installation: from pip install gensim
from gensim import corpora, models, similarities 

In [2]:
#define api_keys, authorization and api object
api_key = ""
api_secret = ""
auth = tweepy.AppAuthHandler(api_key, api_secret)
api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Define Keywords for Data Retrieval

In [25]:
#keyword = "#yoga"
#keyword = "#pizza"
keyword = "#food"

yesterday = "2018-04-08"
today = "2018-04-17" 

file_name = keyword + "-" + today + ".csv"
print("file name:",file_name)

if "-" in keyword:
    keyterm = keyword.split("-")
else:
    keyterm = keyword
print("keyterm:",keyterm)

file name: #food-2018-04-17.csv
keyterm: #food


In [26]:

c = tweepy.Cursor(api.search,q=keyterm,since=yesterday, until=today,lang="en")


In [27]:
tweets = []
for tweet in c.items(10000):
    parsed_tweet = {}
    parsed_tweet['id'] = str(tweet.id)
    parsed_tweet['date'] = tweet.created_at
    parsed_tweet['source'] = tweet.source
    parsed_tweet['text'] = tweet.text
    parsed_tweet['favorites'] = str(tweet.favorite_count)
    parsed_tweet['retweets'] = str(tweet.retweet_count)
    parsed_tweet['user_name'] = tweet.user.name
    parsed_tweet['user_flwr_count'] = str(tweet.user.followers_count)
    if parsed_tweet['favorites'].isdigit() & parsed_tweet['retweets'].isdigit() & parsed_tweet['user_flwr_count'].isdigit() & parsed_tweet['id'].isdigit():
        tweets.append(parsed_tweet)
df = pd.DataFrame(tweets)

Rate limit reached. Sleeping for: 818


In [24]:
df.tail()

,date,favorites,id,retweets,source,text,user_flwr_count,user_name
4461,2018-04-08 01:20:19,3,982790206779789313,1,Twitter for Android,This Low- #Sodium #Pasta Recipe Is Packed With...,254,SMB
4462,2018-04-08 01:04:49,0,982786307968700416,0,Facebook,Hi everyone please check out our new video! T...,126,Samantha Verdaguer
4463,2018-04-08 00:59:48,0,982785042572247040,165,Twitter for Android,"RT @tastemade: If you're craving lasagna, but ...",191,Valen
4464,2018-04-08 00:57:32,0,982784472935534593,6,Twitter for Android,RT @DiplyDelicious: What's for dinner tonight?...,202,Charity Radebe
4465,2018-04-08 00:56:48,2,982784290428776451,0,Instagram,#homecooking #pasta #aldi #aldifinds #italianf...,170,Ros Krasny


In [14]:
df.to_csv(file_name, index=False,header=False)

In [15]:
columns = ['date','favorites','id','retweets','source','text','user_flwr_count','user_name']
data = pd.read_csv(file_name,header = None,names = columns,parse_dates=['date'],dtype={'id':str,'favorites':str,'retweets':str,'user_flwr_count':str})
new_df = data.dropna(subset=['id','favorites','retweets','user_flwr_count'])
new_df.to_csv(file_name, index=False,header=False) 

data = pd.read_csv(file_name,header = None,names = columns,parse_dates=['date'],dtype={'id':str,'favorites':int,'retweets':int,'user_flwr_count':int})

In [16]:
data.tail()

,date,favorites,id,retweets,source,text,user_flwr_count,user_name
9995,2018-04-11 03:20:51,0,983907703474008065,83,Twitter Web Client,RT @GrumpysHoneyB: What are you having on your...,10584,Prime Beauty Blog
9996,2018-04-11 03:16:35,0,983906628985589760,127,My Scheduler Retweets,RT @DomesticSup: This Southwest Pizza is loade...,2529,Chanel van Reenen
9997,2018-04-11 03:15:49,0,983906437381341184,0,Instagram,Sometimes I crave for a #pizza #pizzalover #fo...,1671,Harif Salam
9998,2018-04-11 03:15:42,0,983906408801292289,0,Instagram,"Yes, I am tracing this #AMAZING #ChickenAlfred...",1485,Jeremy Petrachonis
9999,2018-04-11 03:15:34,0,983906373732745221,1,Beer and Pizza - Rewteet,RT @party_on_cheers: Join 7 people right now a...,2755,Beer and Pizza Day!
